In [8]:
from langchain_community.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

import os
from dotenv import load_dotenv, find_dotenv
import fitz

import json

In [4]:
_ = load_dotenv(find_dotenv())
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [6]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = '\n'.join([page.get_text() for page in doc])
    return text.strip()

In [ ]:
class VirtualHRD:
    def __init__(self, position, salary, company_type, cv_path):
        self.chat_model = ChatOpenAI(model='gpt-4o-mini')
        self.position = position
        self.salary = salary
        self.company_type = company_type
        self.cv_text = extract_text_from_pdf(cv_path)
        self.chat_history = [
            SystemMessage(content=f"""
                Anda adalah seorang HRD di perusahaan {self.company_type}.
                Saat ini Anda sedang melakukan wawancara untuk posisi {self.position} dengan gaji {self.salary}.
                CV kandidat telah diberikan berikut:
                {self.cv_text}
                
                Tanyakan pertanyaan secara proaktif, mulai dari pengenalan diri, pengalaman, hingga pertanyaan teknis.
                Setelah cukup, akhiri wawancara dengan memberikan evaluasi & skor (0-100).
                dan kasih output is_finish jadi true, jika dirasa sudah cukup wawancaranya.
                kamu akan bertanya maksimal 10 pertanyaan.

                format output yang harus diberikan dalam JSON:
                {{
                    "response": "<jawaban AI>",
                    "is_finish": <true/false>
                }}
            """)
        ]
    
    def ask_question(self):
        response = self.chat_model.invoke(self.chat_history)

        try:
            response_data = json.loads(response.content)
            response_text = response_data.get("response")
            is_finish = response_data.get("is_finish", False)
        except json.JSONDecodeError:
            response_text = response.content
            is_finish = False

        self.chat_history.append(HumanMessage(content=response_text))
        print(f"\nHRD: {response_text}")
        return response_text, is_finish
    
    def answer_question(self, user_input):
        self.chat_history.append(HumanMessage(content=user_input))

    def init_interview(self):
        print("\n🤖 HRD: Halo! Selamat datang di wawancara untuk posisi Junior Frontend Engineer di perusahaan kami. Bisa perkenalkan diri Anda?\n")
        self.chat_history.append(AIMessage(content="Halo! Selamat datang di wawancara untuk posisi Junior Frontend Engineer di perusahaan kami. Bisa perkenalkan diri Anda?"))
        pass

    def evaluate(self):
        evaluation_prompt = """
        Berdasarkan percakapan sejauh ini dan informasi dari CV, evaluasi apakah wawancara sudah cukup.
        Jika sudah cukup, berikan respons wawancara terakhir dan akhiri dengan pernyataan penutup.
        Jika belum cukup, lanjutkan dengan pertanyaan lain yang relevan dengan posisi yang dilamar.
        Jawaban harus profesional tetapi tetap ramah.

        perhatikan, sudah bertanya kali, dan ingat batas pertanyaannya ada berapa.
        dan format menjawab jangan lupa seperti ini

        """

        self.chat_history.append(SystemMessage(content=evaluation_prompt))

    def run_interview(self):

        self.init_interview()        

        while True:

            user_input = input("\n Kandidat : ")
            self.answer_question(user_input)

            self.evaluate()

            question, is_finish = self.ask_question()
            # print(question)
            if is_finish:
                break

            pass

In [ ]:
cv_path = 'public/CV_Reza Ali Nirwansyah (2).pdf'
virtual_hrd = VirtualHRD(
                position='frontend Developer',
                salary='Rp 4.000.000',
                company_type='non-profit',
                cv_path=cv_path
            )

virtual_hrd.run_interview()


🤖 HRD: Halo! Selamat datang di wawancara untuk posisi Junior Frontend Engineer di perusahaan kami. Bisa perkenalkan diri Anda?


HRD: Terima kasih, Rezza, atas perkenalannya. Selanjutnya, bisa ceritakan sedikit tentang pengalaman Anda sebagai Front-End Developer dalam proyek Chatbot SDPPI - Kominfo? Apa tantangan terbesar yang Anda hadapi dan bagaimana Anda mengatasinya?
Terima kasih, Rezza, atas perkenalannya. Selanjutnya, bisa ceritakan sedikit tentang pengalaman Anda sebagai Front-End Developer dalam proyek Chatbot SDPPI - Kominfo? Apa tantangan terbesar yang Anda hadapi dan bagaimana Anda mengatasinya?

HRD: Terima kasih atas jawaban Anda, Rezza. Sangat menarik mendengar pengalaman Anda dalam proyek tersebut. Selanjutnya, dapatkah Anda menjelaskan teknologi dan alat apa yang Anda gunakan dalam proyek tersebut, serta bagaimana Anda memastikan bahwa antarmuka yang Anda buat responsif dan ramah pengguna?
Terima kasih atas jawaban Anda, Rezza. Sangat menarik mendengar pengalaman Anda 